In [7]:
import pandas as pd
#To see all rows and columns
pd.set_option('display.max_rows', None)

import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
import matplotlib.pylab as plt
from sklearn import preprocessing

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss
from sklearn.metrics import f1_score

from sklearn.metrics import mean_absolute_error 
from sklearn.metrics import mean_squared_error   #RMSE


import itertools 

from pathlib import Path

from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import BaggingRegressor

from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import average_precision_score
from sklearn.metrics import classification_report


from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier

In [2]:
#Problem 4
ebay_data = pd.read_csv("/Users/nivethida/Documents/Spring Sem/DataMining/Classification/eBayAuctions.csv")

In [3]:
ebay_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1972 entries, 0 to 1971
Data columns (total 8 columns):
Category        1972 non-null object
currency        1972 non-null object
sellerRating    1972 non-null int64
Duration        1972 non-null int64
endDay          1972 non-null object
ClosePrice      1972 non-null float64
OpenPrice       1972 non-null float64
Competitive?    1972 non-null int64
dtypes: float64(2), int64(3), object(3)
memory usage: 123.4+ KB


In [4]:
x = ebay_data.drop(columns=['Competitive?'])
x = pd.get_dummies(x)
y = ebay_data['Competitive?']
train_X, valid_X, train_y, valid_y = train_test_split(x, y, test_size=0.40, random_state=3)

In [8]:
def eval_tp(actual,propensity,cutoff):
    
    cm = confusion_matrix(actual,[1 if p > cutoff else 0 for p in propensity]).ravel()
    
    tn=cm[0]
    fp=cm[1]
    fn=cm[2]
    tp=cm[3]
    true_positive_rate = tp / (tp + fn)
    sensitivity = true_positive_rate
    recall  = true_positive_rate
    hit_rate = true_positive_rate
    precision= tp / (tp + fp)
    positive_predictive_value=precision

    roc_auc_scor=roc_auc_score(actual,[1 if p > cutoff else 0 for p in propensity])
    
    return "true negative "+ str(tn), \
           "false positive "+str(fp),\
           "false negative "+str(fn),\
           "true positive "+str(tp),\
           "true positive rate "+str(true_positive_rate),\
           "sensitivity "+str(sensitivity),\
           "recall "+str(recall),\
           "hit rate "+str(hit_rate),\
           "precision "+str(precision),\
           "positive predictive value " + str(positive_predictive_value)


# DecisionTreeClassifier

In [21]:
clf = DecisionTreeClassifier(random_state=1)

param_grid = {}

gridSearch = GridSearchCV(clf, param_grid, cv=5, n_jobs=-1)
gridSearch.fit(train_X, train_y)

clf = gridSearch.best_estimator_

print(clf)
                  
print(eval_tp(actual=valid_y,
        propensity=pd.DataFrame(clf.predict_proba(valid_X)).iloc[:,1],
        cutoff=0.5))

tree_reg_proba = clf.predict_proba(valid_X)

df_valid=pd.DataFrame()
df_valid['class']=valid_y
df_valid['prob_0']=tree_reg_proba[:,0]
df_valid['prob_1']=tree_reg_proba[:,1]
df_valid['y_pred']=[1 if p > 0.5 else 0 for p in df_valid.prob_1]
df_valid
print('-'*50)
print("accuracy")
print(accuracy_score(df_valid['class'],df_valid['y_pred']))

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=1, splitter='best')
('true negative 312', 'false positive 43', 'false negative 64', 'true positive 370', 'true positive rate 0.8525345622119815', 'sensitivity 0.8525345622119815', 'recall 0.8525345622119815', 'hit rate 0.8525345622119815', 'precision 0.8958837772397095', 'positive predictive value 0.8958837772397095')
--------------------------------------------------
accuracy
0.8643852978453739


# AdaBoostClassifier with DecisionTreeClassifier

In [22]:
clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=6,min_samples_split=2,
                                               min_samples_leaf=5,max_features=10,random_state=1), 
                                               n_estimators=100, random_state=1)

param_grid = {}
gridSearch = GridSearchCV(clf, 
                          param_grid, cv=5, n_jobs=-1)
gridSearch.fit(train_X, train_y)

clf = gridSearch.best_estimator_

print(clf)
                        
print(eval_tp(actual=valid_y,
        propensity=pd.DataFrame(clf.predict_proba(valid_X)).iloc[:,1],
        cutoff=0.5))

adabooster_reg_proba = clf.predict_proba(valid_X)

df_valid=pd.DataFrame()
df_valid['class']=valid_y
df_valid['prob_0']=adabooster_reg_proba[:,0]
df_valid['prob_1']=adabooster_reg_proba[:,1]
df_valid['y_pred']=[1 if p > 0.5 else 0 for p in df_valid.prob_1]
df_valid
print('-'*50)
print("accuracy")
print(accuracy_score(df_valid['class'],df_valid['y_pred']))

AdaBoostClassifier(algorithm='SAMME.R',
                   base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                         class_weight=None,
                                                         criterion='gini',
                                                         max_depth=6,
                                                         max_features=10,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=5,
                                                         min_samples_split=2,
                                                         min_weight_fraction_leaf=0.0,
                                                         presort='deprecated',
                            

# BaggingClassifier with DecisionTreeClassifier

In [23]:
clf = BaggingClassifier(DecisionTreeClassifier(max_depth=6,min_samples_split=2,
                                               min_samples_leaf=5,max_features=10,random_state=1), 
                                               n_estimators=100, random_state=1)

param_grid = {}
gridSearch = GridSearchCV(clf, 
                          param_grid, cv=5, n_jobs=-1)
gridSearch.fit(train_X, train_y)

clf = gridSearch.best_estimator_

print(clf)

print(eval_tp(actual=valid_y,
        propensity=pd.DataFrame(clf.predict_proba(valid_X)).iloc[:,1],
        cutoff=0.5))

bagging_reg_proba = clf.predict_proba(valid_X)

df_valid=pd.DataFrame()
df_valid['class']=valid_y
df_valid['prob_0']=bagging_reg_proba[:,0]
df_valid['prob_1']=bagging_reg_proba[:,1]
df_valid['y_pred']=[1 if p > 0.5 else 0 for p in df_valid.prob_1]
df_valid
print('-'*50)
print("accuracy")
print(accuracy_score(df_valid['class'],df_valid['y_pred']))

BaggingClassifier(base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='gini',
                                                        max_depth=6,
                                                        max_features=10,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=5,
                                                        min_samples_split=2,
                                                        min_weight_fraction_leaf=0.0,
                                                        presort='deprecated',
                                                        random_state=1,
        

# RandomForestClassifier

In [24]:
clf = RandomForestClassifier(random_state=1)


param_grid = {}
gridSearch = GridSearchCV(clf, 
                          param_grid, cv=5, n_jobs=-1,)
gridSearch.fit(train_X, train_y)

clf = gridSearch.best_estimator_

print(clf)

print(eval_tp(actual=valid_y,
        propensity=pd.DataFrame(clf.predict_proba(valid_X)).iloc[:,1],
        cutoff=0.5))

forest_reg_proba = clf.predict_proba(valid_X)

df_valid=pd.DataFrame()
df_valid['class']=valid_y
df_valid['prob_0']=forest_reg_proba[:,0]
df_valid['prob_1']=forest_reg_proba[:,1]
df_valid['y_pred']=[1 if p > 0.5 else 0 for p in df_valid.prob_1]
df_valid
print('-'*50)
print("accuracy")
print(accuracy_score(df_valid['class'],df_valid['y_pred']))


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=1, verbose=0,
                       warm_start=False)
('true negative 312', 'false positive 43', 'false negative 80', 'true positive 354', 'true positive rate 0.815668202764977', 'sensitivity 0.815668202764977', 'recall 0.815668202764977', 'hit rate 0.815668202764977', 'precision 0.8916876574307305', 'positive predictive value 0.8916876574307305')
--------------------------------------------------
accuracy
0.844106463878327


BaggingClassifier accuracy is less( 0.826362484157161 ) when compared with RandomForestClassifier which is 0.844106463878327

# GradientBoostingClassifier

In [25]:
clf = GradientBoostingClassifier()

param_grid = {}

gridSearch = GridSearchCV(clf, 
                          param_grid, cv=5, n_jobs=-1)
gridSearch.fit(train_X, train_y)

clf = gridSearch.best_estimator_

print(clf)

print(eval_tp(actual=valid_y,
        propensity=pd.DataFrame(clf.predict_proba(valid_X)).iloc[:,1],
        cutoff=0.5))

boosting_reg_proba = clf.predict_proba(valid_X)

df_valid=pd.DataFrame()
df_valid['class']=valid_y
df_valid['prob_0']=boosting_reg_proba[:,0]
df_valid['prob_1']=boosting_reg_proba[:,1]
df_valid['y_pred']=[1 if p > 0.5 else 0 for p in df_valid.prob_1]
df_valid
print('-'*50)
print("accuracy")
print(accuracy_score(df_valid['class'],df_valid['y_pred']))

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)
('true negative 333', 'false positive 22', 'false negative 78', 'true positive 356', 'true positive rate 0.8202764976958525', 'sensitivity 0.8202764976958525', 'recall 0.8202764976958525', 'hit rate 0.8202764976958525', 'precision 0.9417989417989417', 'positive predictive value 0.9417989417989417')
---

GradientBoostingClassifier has higher accuracy 0.8732572877059569 when compared with BaggingClassifier accuracy and RandomForestClassifier